In [1]:
from math import e, factorial,log, gamma, sqrt, floor, exp
import re
from matplotlib import pyplot as pt
from analysis import *

In [2]:
f1 = open("r2-d0-result.txt")
raw1 = f1.read()
f1.close()

f2 = open("r2-d1-result.txt")
raw2 = f2.read()
f2.close()

f3 = open("r5-d0-result.txt")
raw3 = f3.read()
f3.close()

f4 = open("r5-d1-result.txt")
raw4 = f4.read()
f4.close()

f5 = open("r7-d0-result.txt")
raw5 = f5.read()
f5.close()

f6 = open("r7-d1-result.txt")
raw6 = f6.read()
f6.close()

f7 = open("r9-d0-result.txt")
raw7 = f7.read()
f7.close()

f8 = open("r9-d1-result.txt")
raw8 = f8.read()
f8.close()

Ttx = 23
Trx = 60
Tn = 100

In [3]:
Eb1,Es1,En1 = computeEvents(raw1,Ttx,Trx,Tn)
Eb2,Es2,En2 = computeEvents(raw2,Ttx,Trx,Tn)
Eb3,Es3,En3 = computeEvents(raw3,Ttx,Trx,Tn)
Eb4,Es4,En4 = computeEvents(raw4,Ttx,Trx,Tn)
Eb5,Es5,En5 = computeEvents(raw5,Ttx,Trx,Tn)
Eb6,Es6,En6 = computeEvents(raw6,Ttx,Trx,Tn)
Eb7,Es7,En7 = computeEvents(raw7,Ttx,Trx,Tn)
Eb8,Es8,En8 = computeEvents(raw8,Ttx,Trx,Tn)

In [14]:
Pb1 = Eb1*Ttx/(Eb1*Ttx+En1*Tn+Es1*Trx)
Ps1 = Es1*Trx/(Eb1*Ttx+En1*Tn+Es1*Trx)

Pb2 = Eb2*Ttx/(Eb2*Ttx+En2*Tn+Es2*Trx)
Ps2 = 1-Pb2

Pb3 = Eb3*Ttx/(Eb3*Ttx+En3*Tn+Es3*Trx)
Ps3 = 1-Pb3

Pb4 = Eb4*Ttx/(Eb4*Ttx+En4*Tn+Es4*Trx)
Ps4 = 1-Pb4

Pb5 = Eb5*Ttx/(Eb5*Ttx+En5*Tn+Es5*Trx)
Ps5 = 1-Pb5

Pb6 = Eb6*Ttx/(Eb6*Ttx+En6*Tn+Es6*Trx)
Ps6 = 1-Pb6

Pb7 = Eb7*Ttx/(Eb7*Ttx+En7*Tn+Es7*Trx)
Ps7 = 1-Pb7

Pb8 = Eb8*Ttx/(Eb8*Ttx+En8*Tn+Es8*Trx)
Ps8 = 1-Pb8

print(Ps1*Pb2)
print(Ps3*Pb4)
print(Ps5*Pb6)
print(Ps7*Pb8)

0.16077692208401304
0.2509217686168981
0.20953350031758142
0.09014361480213014


In [ ]:
wnds1 = genWindowHistogram(raw1,raw2,1000,Trx)
wnds2 = genWindowHistogram(raw3,raw4,1000,Trx)
wnds3 = genWindowHistogram(raw5,raw6,1000,Trx)
wnds4 = genWindowHistogram(raw7,raw8,1000,Trx)

Psuc1 = checkSuccess(raw2,raw1)*Ttx/(Eb1*Ttx+En1*Tn+Es1*Trx)
Psuc2 = checkSuccess(raw4,raw3)*Ttx/(Eb3*Ttx+En3*Tn+Es3*Trx)
Psuc3 = checkSuccess(raw6,raw5)*Ttx/(Eb5*Ttx+En5*Tn+Es5*Trx)
Psuc4 = checkSuccess(raw8,raw7)*Ttx/(Eb7*Ttx+En7*Tn+Es7*Trx)
#pois_d1 = poisson_dist(Trx*Eb2/len(raw1),len(wnds1))
#pois_d2 = poisson_dist(Trx*Eb4/len(raw3),len(wnds2))
#pois_d3 = poisson_dist(Trx*Eb6/len(raw5),len(wnds3))

1.0000000000000002
0.9999999999999999
1.0
1.0


In [ ]:
pt.rcParams["figure.figsize"] = (16,10)

pt.plot(range(len(wnds1)),wnds1,label=r"$P_{S}=%.3f;~P_{B}=%.3f;~P_{Succ}$= %.3f"%(Ps1,Pb2,Psuc1))
pt.plot(range(len(wnds2)),wnds2,label=r"$P_{S}=%.3f;~P_{B}=%.3f;~P_{Succ}$= %.3f"%(Ps3,Pb4,Psuc2))
pt.plot(range(len(wnds3)),wnds3,label=r"$P_{S}=%.3f;~P_{B}=%.3f;~P_{Succ}$= %.3f"%(Ps5,Pb6,Psuc3))
pt.plot(range(len(wnds4)),wnds4,label=r"$P_{S}=%.3f;~P_{B}=%.3f;~P_{Succ}$= %.3f"%(Ps7,Pb8,Psuc4))
pt.legend(prop={'family':'Times New Roman', 'size':20})
#pt.plot(range(len(wnds1)),pois_d1)
#pt.plot(range(len(wnds1)),pois_d2)
#pt.plot(range(len(wnds1)),pois_d3)
pt.axis([0,25,0,0.2])
pt.xticks(range(25), [str(int(n)) for n in range(25)])
pt.xlabel(r'$\mathcal{k}$', fontsize = 18)
pt.ylabel(r'P(k-messages-received)')
pt.grid(True)
pt.show()